##Tomando 2018 y 2019

In [ ]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1622213870570_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1622213870570_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1622213870570_0002,pyspark,idle,Link,Link,✔


In [ ]:
sc.install_pypi_package("pandas") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository
sc.install_pypi_package("s3fs") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Running setup.py bdist_wheel for subprocess32: started
  Running setup.py bdist_wheel for subprocess32: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built subprocess32
  Found existing installation: pyparsing 1.5.6
    Not uninstalling pyparsing at /usr/lib/python2.7/site-packages, outside environment /tmp/1622215631292-0
  Found existing installation: six 1.9.0
    Not uninstalling six at /usr/lib/python2.7/site-packages, outside environment /tmp/1622215631292-0

  Running setup.py bdist_wheel for s3fs: started
  Running setup.py bdist_wheel for s3fs: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/94/20/c9/b0256fe423de22bca83518fe0f7eba34ae1b9591a9d8c2d3b4
Successfully built s3fs
  Found existing installation: s3transfer 0.3.3
    Not uninstalling s3transfer at /usr/lib/python2.7/site-packages, outside environment /tmp/1622215631292-0

In [ ]:
#input_bucket = 's3://rawtaxisjara/taxis_data/'
#input_path = '*.csv'
#df = spark.read.csv(input_bucket + input_path, header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#input_bucket = 's3://taxidatamineria/'
#input_path = 'taxi_zone_lookup.csv'
#df_zone = spark.read.csv(input_bucket + input_path, header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Para leer todos los datos
input_bucket = 's3://taxidatamineria/yellow_trip_data/'
input_path = '*.csv'
df = spark.read.csv(input_bucket + input_path, header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Para leer todos los datos
input_bucket = 's3://taxidatamineria/'
input_path = 'taxi_zone_lookup.csv'
df_zone = spark.read.csv(input_bucket + input_path, header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df = df.join(df_zone, df.PULocationID == df_zone.LocationID,how='left')
df = df.withColumnRenamed("Borough", "pu_borough")
df = df.withColumnRenamed("Zone", "pu_zone")
df = df.withColumnRenamed("service_zone", "pu_service_zone")
df = df.drop("LocationID")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df = df.join(df_zone, df.DOLocationID == df_zone.LocationID,how='left')
df = df.withColumnRenamed("Borough", "do_borough")
df = df.withColumnRenamed("Zone", "do_zone")
df = df.withColumnRenamed("service_zone", "do_service_zone")
df = df.drop("LocationID")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df.count() #53.925.735

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df1, names, newType):
    """
    df1     : DataFrame with the columns of data
    names  : List containing the names of all the columns that will be converted
    newType: New data type
    """
    for name in names: 
        df1 = df1.withColumn(name, df[name].cast(newType))
    return df1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Convert the df columns to datetime()
df = df.withColumn("tpep_pickup_datetime",to_timestamp(df.tpep_pickup_datetime)) \
  .withColumn("tpep_dropoff_datetime", to_timestamp(df.tpep_dropoff_datetime))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Convert all features listed to `IntegerType()`
df = convertColumn(df, ['Passenger_count'], IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#%%time
# List with the names of all the features to be converted to FloatType

columns = ['trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

# Convert all features in `columns` to `FloatType()`
df = convertColumn(df, columns, FloatType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Convert all features listed to `StringType()`
df = convertColumn(df, ['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type'], StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Calculate Time difference in Mins
df=df.withColumn('elapsed_time',round((col('tpep_dropoff_datetime').cast('long') - col('tpep_pickup_datetime').cast('long'))/60))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df = df.na.drop(how='any')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def taxi_data_cleaner(data):
    
    
    #Dejar en donde total_amount > 0 y < 500 USD
    data = data.filter(data.total_amount > 0)
    data = data.filter(data.total_amount < 1000)

    #Dejar los que trip_distance > 0 y <= 20 millas
    data = data.filter(data.trip_distance >= 0)
    data = data.filter(data.trip_distance <= 20) #OJO Eran 150

    #Dejar donde passenger_count <= 5
    data = data.filter(data.Passenger_count <= 5)
     
    #Dejar fare_amount > 0 USD < 1000 USD
    data = data.filter(data.fare_amount > 0)
    data = data.filter(data.fare_amount < 1000)

    #Dejar extra > 0 
    data = data.filter(data.extra >= 0)

    #Dejar tip_amount > 0 
    data = data.filter(data.tip_amount >= 0)
    
    #Dejar mta_tax > 0 
    data = data.filter(data.mta_tax >= 0)

    #Dejar tolls_amount > 0 
    data = data.filter(data.tolls_amount >= 0)

    #Dejar elapsed_time > 0 y <=81 mins
    data = data.filter(data.elapsed_time > 0)
    data = data.filter(data.elapsed_time <= 81)

 
    # Adicionar columnas de tiempo
    data = data.withColumn('pu_year', year('tpep_pickup_datetime'))\
      .withColumn('pu_month', month('tpep_pickup_datetime'))\
      .withColumn('do_month', month('tpep_dropoff_datetime'))\
      .withColumn('pu_day', dayofmonth('tpep_pickup_datetime'))\
      .withColumn('do_day', dayofmonth('tpep_dropoff_datetime'))\
      .withColumn('pu_hour', hour('tpep_pickup_datetime'))\
      .withColumn('do_hour', hour('tpep_dropoff_datetime'))\
      .withColumn('pu_dayname', date_format('tpep_pickup_datetime', 'EEEE'))\
      .withColumn('do_dayname', date_format('tpep_dropoff_datetime', 'EEEE'))\
      .withColumn('pu_daynum', dayofweek('tpep_pickup_datetime')-1) #lunes = 1    
    
    #Adicionar col hora pico
    data = data.withColumn('peak_hour',\
       when((col('pu_hour') >= 8 ) & (col('pu_hour') <= 10) & (col('pu_daynum') < 6), 'Pico_AM')\
      .when((col('pu_hour') >= 16) & (col('pu_hour') <= 19) & (col('pu_daynum') < 6), 'Pico_PM')\
      .otherwise('No_pico'))
    
    return(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Llamamos la función de limpieza y creación de variables
df = taxi_data_cleaner(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#--------------------Sacamos muestra para trabajar modelos 2019

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_limit = df.limit(1000) #MIL PINCHES REGISTROS

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_2018.count() #99.307.515

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_2018.write.save("s3://rawtaxisjara/sample2/sample_2018.csv", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_muestra = df.sample(fraction=0.01, seed=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_muestra.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#"Hasta acá"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df.write.save("s3://taxidatamineria/yellow_trip_data/Outputs/all_data_models.csv", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector

from pyspark.sql import SQLContext
from pyspark.sql import types
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_data = df_limit[["trip_distance", "pu_borough","do_borough","total_amount","pu_hour","pu_dayname","elapsed_time","peak_hour"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_data = df[["trip_distance", "pu_borough","do_borough","total_amount","pu_hour","pu_dayname","elapsed_time","peak_hour"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
no_dummies_df = df_data[["trip_distance", "total_amount","pu_hour","elapsed_time"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_data_dummies = df_data[["pu_borough","do_borough","pu_dayname","peak_hour"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import itertools

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
'''
def spark_get_dummies(df):
    
    categories = []
    for i, values in enumerate(df.columns):
        categories.append(df.select(values).distinct().rdd.flatMap(lambda x: x).collect())
        
    expressions = []
    for i, values in enumerate(df.columns):
        expressions.append([F.when(F.col(values) == i, 1).otherwise(0).alias(str(values) + "_" + str(i)) for i in categories[i]])
    
    expressions_flat = list(itertools.chain.from_iterable(expressions))
    
    df_final = df.select(*expressions_flat)
    
    return df_final
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\ndef spark_get_dummies(df):\n\n    categories = []\n    for i, values in enumerate(df.columns):\n        categories.append(df.select(values).distinct().rdd.flatMap(lambda x: x).collect())\n\n    expressions = []\n    for i, values in enumerate(df.columns):\n        expressions.append([F.when(F.col(values) == i, 1).otherwise(0).alias(str(values) + "_" + str(i)) for i in categories[i]])\n\n    expressions_flat = list(itertools.chain.from_iterable(expressions))\n\n    df_final = df.select(*expressions_flat)\n\n    return df_final\n'

In [ ]:
#dummies_df = spark_get_dummies(df_data_dummies)
#dummies_df.show(10) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
'''
import pyspark.sql.functions as F
df = sqlContext.createDataFrame([
    (1, "A", "X1"),
    (2, "B", "X2"),
    (3, "B", "X3"),
    (1, "B", "X3"),
    (2, "C", "X2"),
    (3, "C", "X2"),
    (1, "C", "X1"),
    (1, "B", "X1"),
], ["ID", "TYPE", "CODE"])

types = df.select("TYPE").distinct().rdd.flatMap(lambda x: x).collect()
codes = df.select("CODE").distinct().rdd.flatMap(lambda x: x).collect()
types_expr = [F.when(F.col("TYPE") == ty, 1).otherwise(0).alias("e_TYPE_" + ty) for ty in types]
codes_expr = [F.when(F.col("CODE") == code, 1).otherwise(0).alias("e_CODE_" + code) for code in codes]
df = df.select("ID", "TYPE", "CODE", *types_expr+codes_expr)
df.show()
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'\nimport pyspark.sql.functions as F\ndf = sqlContext.createDataFrame([\n    (1, "A", "X1"),\n    (2, "B", "X2"),\n    (3, "B", "X3"),\n    (1, "B", "X3"),\n    (2, "C", "X2"),\n    (3, "C", "X2"),\n    (1, "C", "X1"),\n    (1, "B", "X1"),\n], ["ID", "TYPE", "CODE"])\n\ntypes = df.select("TYPE").distinct().rdd.flatMap(lambda x: x).collect()\ncodes = df.select("CODE").distinct().rdd.flatMap(lambda x: x).collect()\ntypes_expr = [F.when(F.col("TYPE") == ty, 1).otherwise(0).alias("e_TYPE_" + ty) for ty in types]\ncodes_expr = [F.when(F.col("CODE") == code, 1).otherwise(0).alias("e_CODE_" + code) for code in codes]\ndf = df.select("ID", "TYPE", "CODE", *types_expr+codes_expr)\ndf.show()\n'

In [ ]:
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pu_borough = df_data_dummies.select("pu_borough").distinct().rdd.flatMap(lambda x: x).collect()
pu_dayname = df_data_dummies.select("pu_dayname").distinct().rdd.flatMap(lambda x: x).collect()
pu_borough_expr = [F.when(F.col("pu_borough") == ty, 1).otherwise(0).alias("pu_borough_" + ty) for ty in pu_borough]
pu_dayname_expr = [F.when(F.col("pu_dayname") == code, 1).otherwise(0).alias("pu_dayname_" + code) for code in pu_dayname]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
do_borough = df_data_dummies.select("do_borough").distinct().rdd.flatMap(lambda x: x).collect()
peak_hour = df_data_dummies.select("peak_hour").distinct().rdd.flatMap(lambda x: x).collect()
do_borough_expr = [F.when(F.col("do_borough") == ty, 1).otherwise(0).alias("do_borough_" + ty) for ty in do_borough]
peak_hour_expr = [F.when(F.col("peak_hour") == code, 1).otherwise(0).alias("peak_hour_" + code) for code in peak_hour]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df3 = df_data_dummies.select(*pu_borough_expr+pu_dayname_expr+do_borough_expr+peak_hour_expr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
no_dummies_df = no_dummies_df.withColumn("id", monotonically_increasing_id())
dummies_df = df3.withColumn("id", monotonically_increasing_id())
X = dummies_df.join(no_dummies_df, "id", "outer").drop("id")
#X.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df3 = df_data_dummies.select("pu_borough", "pu_dayname",*types_expr+codes_expr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#no_dummies_df = no_dummies_df.withColumn("id", monotonically_increasing_id())
#dummies_df = dummies_df.withColumn("id", monotonically_increasing_id())
#X = dummies_df.join(no_dummies_df, "id", "outer").drop("id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
X.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- pu_borough_EWR: integer (nullable = true)
 |-- pu_borough_Brooklyn: integer (nullable = true)
 |-- pu_borough_Manhattan: integer (nullable = true)
 |-- pu_borough_Bronx: integer (nullable = true)
 |-- pu_borough_Queens: integer (nullable = true)
 |-- pu_borough_Unknown: integer (nullable = true)
 |-- pu_borough_Staten Island: integer (nullable = true)
 |-- pu_dayname_Wednesday: integer (nullable = true)
 |-- pu_dayname_Saturday: integer (nullable = true)
 |-- pu_dayname_Tuesday: integer (nullable = true)
 |-- pu_dayname_Monday: integer (nullable = true)
 |-- pu_dayname_Sunday: integer (nullable = true)
 |-- pu_dayname_Friday: integer (nullable = true)
 |-- pu_dayname_Thursday: integer (nullable = true)
 |-- do_borough_EWR: integer (nullable = true)
 |-- do_borough_Brooklyn: integer (nullable = true)
 |-- do_borough_Manhattan: integer (nullable = true)
 |-- do_borough_Bronx: integer (nullable = true)
 |-- do_borough_Queens: integer (nullable = true)
 |-- do_borough_Unknown: in

In [ ]:
#Los resultados son mejores con total amount, pero la verdad considero que poner esa variable es como hacer trampa 'total_amount'
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['pu_borough_Queens',
                                               'do_borough_EWR',
                                               'do_borough_Unknown',
                                               'do_borough_Brooklyn',
                                               'pu_dayname_Thursday',
                                               'pu_dayname_Monday',
                                               'pu_dayname_Sunday',
                                               'peak_hour_No_pico',
                                               'trip_distance'], outputCol = 'features')
vX = vectorAssembler.transform(X)
vX = vX.select(['features', 'elapsed_time'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
'''
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['pu_borough_Queens',
                                               'pu_borough_EWR',
                                               'pu_borough_Unknown',
                                               'pu_borough_Brooklyn',
                                               'pu_borough_Staten Island',
                                               'pu_borough_Manhattan',
                                               'pu_borough_Bronx',
                                               'do_borough_Queens',
                                               'do_borough_EWR',
                                               'do_borough_Unknown',
                                               'do_borough_Brooklyn',
                                               'do_borough_Staten Island',
                                               'do_borough_Manhattan',
                                               'do_borough_Bronx',
                                               'pu_dayname_Wednesday',
                                               'pu_dayname_Tuesday',
                                               'pu_dayname_Friday',
                                               'pu_dayname_Thursday',
                                               'pu_dayname_Saturday',
                                               'pu_dayname_Monday',
                                               'pu_dayname_Sunday',
                                               'peak_hour_Pico_PM',
                                               'peak_hour_No_pico',
                                               'peak_hour_Pico_AM',
                                               'trip_distance',
                                               'total_amount',
                                               'pu_hour'], outputCol = 'features')
vX = vectorAssembler.transform(X)
vX = vX.select(['features', 'elapsed_time'])
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"\nfrom pyspark.ml.feature import VectorAssembler\nvectorAssembler = VectorAssembler(inputCols = ['pu_borough_Queens',\n                                               'pu_borough_EWR',\n                                               'pu_borough_Unknown',\n                                               'pu_borough_Brooklyn',\n                                               'pu_borough_Staten Island',\n                                               'pu_borough_Manhattan',\n                                               'pu_borough_Bronx',\n                                               'do_borough_Queens',\n                                               'do_borough_EWR',\n                                               'do_borough_Unknown',\n                                               'do_borough_Brooklyn',\n                                               'do_borough_Staten Island',\n                                               'do_borough_Manhattan',\n                                 

In [ ]:
# splitting dataset into train and test set
splits = vX.randomSplit([0.7, 0.3])
train = splits[0] 
test = splits[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
inicio = time.time()
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='elapsed_time', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
fin = time.time()
print(fin-inicio) #

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: [-1.383735848224687,-2.8992848944384435,0.0,0.9446815105525385,0.29463494942135504,-0.4195769698227609,-1.4566007682174822,-1.3429015636721027,2.4775414349757825]
Intercept: 8.31232289242
1904.0601759

In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("ExplainedVariance: %f" % trainingSummary.explainedVariance)
print("r2: %f" % trainingSummary.r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE: 6.563802
ExplainedVariance: 67.926664
r2: 0.631102

In [ ]:
#Predicción regresión lineal
lr_predictions = lr_model.transform(test)
#lr_predictions.select("prediction","elapsed_time","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="elapsed_time",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R Squared (R2) on test data = 0.631063

In [ ]:
test_result = lr_model.evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 6.5625

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="elapsed_time",metricName="mae")
print("mean absolute error (mae) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mean absolute error (mae) on test data = 4.67155

In [ ]:
#Gradient boosted tree regression
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'elapsed_time', maxIter=10)
gbt_model = gbt.fit(train)
gbt_predictions = gbt_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 5.96693

In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="mae")
maegbt = gbt_evaluator.evaluate(gbt_predictions)
print("mean absolute error (mae) on test data = %g" % maegbt)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mean absolute error (mae) on test data = 4.03771

In [ ]:
gbt_evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="r2")
maegbt = gbt_evaluator.evaluate(gbt_predictions)
print("R2 = %g" % r2)

In [ ]:
#Random Forest
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['pu_borough_Queens',
                                               "pu_borough_Manhattan",
                                               "do_borough_Queens",
                                               'do_borough_EWR',
                                               'do_borough_Brooklyn',
                                               "do_borough_Manhattan",
                                               'pu_dayname_Sunday',
                                               "peak_hour_Pico_PM",
                                               'peak_hour_No_pico',
                                               "pu_hour",
                                               'trip_distance'], outputCol = 'features')
vX = vectorAssembler.transform(X)
vX = vX.select(['features', 'elapsed_time'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="elapsed_time", featuresCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
(trainingData, testData) = X.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
model = pipeline.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 5.93743

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="mae")
mae= evaluator.evaluate(predictions)
print("Mean Squared Error (MSE) on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean Squared Error (MSE) on test data = 4.12165

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="elapsed_time", predictionCol="prediction", metricName="r2")
r2= evaluator.evaluate(predictions)
print("R2= %g" % r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R2= 0.69811

In [ ]:
X.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

180553364

In [ ]:
import matplotlib.pyplot as plt
bestPipeline = model
bestModel = bestPipeline.stages[1]
importances = bestModel.featureImportances
x_values = list(range(len(importances)))
plt.figure(figsize=(10, 6))
plt.bar(x_values, importances, orientation="vertical")
plt.xticks(x_values, feature_list, rotation=90)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importances')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Matplotlib requires six>=1.10; you have 1.9.0
Traceback (most recent call last):
  File "/tmp/1622215631292-0/lib/python2.7/site-packages/matplotlib/__init__.py", line 197, in <module>
    "Matplotlib requires six>=1.10; you have %s" % six.__version__)
ImportError: Matplotlib requires six>=1.10; you have 1.9.0



In [ ]:
mean(X['elapsed_time'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column<avg(elapsed_time)>

In [ ]:
type(X)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [ ]:
X.agg({'elapsed_time': 'mean'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
| avg(elapsed_time)|
+------------------+
|14.139237920817692|
+------------------+

In [ ]:
X.agg({'elapsed_time': 'variance'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|variance(elapsed_time)|
+----------------------+
|    116.77201923442081|
+----------------------+

In [ ]:
X.agg({'elapsed_time': 'max'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|max(elapsed_time)|
+-----------------+
|             81.0|
+-----------------+